![logo price](./imgs/massp_logo.png)

## Instructors
* Vương Phúc Thành


#  Bài toán Giới thiệu về pipeline cho một bài toán Machine learning
![home price](./imgs/wine-dataset.jpeg)





###  Mô tả :
Bài toán đưa ra yêu cầu chúng ta phân loại và chấm điểm chất lượng rượu dựa vào các chỉ số thành phần của rượu. 
- Đầu vào: một bảng thông tin tổng quan các chỉ số của rượu
- Đầu ra: chất lượng rượu, đánh số từ 0 đến 10

Mục tiêu: Xây dựng mô hình học máy dựa trên tập training (bao gồm đầu vào và đầu ra) để dự đoán chất lượng của rượu

## Prediction of Wine quality

## Outline

- <a href='#1'>1. Problem defining</a>

- <a href='#2'>2. Khai phá dữ liệu </a>  

- <a href='#3'>3. Data analyzing - Phân tích dữ liệu </a>
    - <a id='#3.1'>3.1. Tổng quan trường target </a>
    - <a id='#3.2'>3.2. Metric đánh giá </a> 
    - <a id='#3.3'>3.3. Khai phá dữ liệu </a> 
- <a href='#4'>4. Feature engineering </a>
    - <a id='#4-1'>4.1. Log transform </a>
    - <a id='#4-2'>4.2. Xử lí imbalanced data</a> 
    - <a id='#4-3'>4.3. Xử lí null data</a> 
- <a href='#5'>5. Modeling and evaluating </a>
  

### <a id='1'>1. Problem defining

 Chúng ta sẽ sử dụng model classification để phân loại rượu vào thang điểm từ 0 - 10. Đối với bài classification sẽ là 10 classes.

### <a id='2'>2. Data collecting

Đối với bài toán này ta đã có sẵn 1 tập data trong file csv, có thể load và sử dụng ngay, không cần qua các bước thu thập và load vào database.

In [1]:
## import các thư viện cần thiết
import os  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error

from IPython.display import Image

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("data/winequalityN.csv")
df.head(10)

### <a id='3'> 3. Data analyzing

In [ ]:
## Một vài thông tin tổng quan tập dữ liệu:
print("Kích thước tập dữ liệu: {}".format(df.shape))

#### <a id='3.1'> 3.1 Tổng quan trường target (quality)


In [ ]:
df['quality'].describe()

In [ ]:
## phân bố cột dữ liệu quality
sns.countplot(df['quality'])

Nhận xét: số lượng rượu với quality 5 và 6 lớn hơn nhiều lần so với các class khác. Nếu ta giữ nguyên phân bố như vậy, có thể sẽ có vấn đề khi dự đoán chất lượng của rượu, do dữ liệu bị imbalanced. 
=> Chúng ta có thể xử lí bằng cách giảm số lượng class 5 và 6 xuống (undersampling) hoặc tăng số lượng các class khác lên (oversampling) 

#### <a id='3.2'> 3.2 Metric đánh giá:

- Đối với 1 bài toán Ordinal Regression, chúng ta có thể sử dùng công thức Mean squared error (Ước lượng trung bình bình phương sai số):


In [ ]:
Image(url='./imgs/MSE.jpg', width=400)

- Đối với 1 bài toán classification, ở đây ta chia label bài toán ra làm 10 classes khác nhau, có thể dùng confusion matrix và f1-score để đánh giá:

In [ ]:
Image(url='./imgs/cm_2.png', width=400)

In [ ]:
Image(url='./imgs/F1-Score.png', width=400)

In [ ]:
#Trong đó
Image(url='./imgs/precision_recall.jpeg', width=400)

#### Đặt câu hỏi và giải đáp về các metrics !?

Chúng ta sẽ nhìn qua bộ dữ liệu, xử lí dữ liệu nhiễu và dữ liệu bị thiếu, kiểm tra type của các cột và điều chỉnh lại cho chuẩn
để sử dụng cho việc phân tích.

In [ ]:
df.info()

In [ ]:
## Số lượng missing values của từng cột 
df.isna().sum()

#### <a id='3.3'>3.3 Data exploratory

Có 12 trường tất cả, vậy việc phân tích dữ liệu sẽ không quá phức tạp, chúng ta sẽ nhìn qua từng trường một. Đầu tiên, nhìn qua phần summary của dataset:

In [ ]:
df.describe()

Đầu tiên cần nhận thấy rằng các trường đang phân bố ở các scale khác nhau, vậy sau này ta sẽ cần rescale lại data.
Có 1 trường object duy nhất, ta sẽ kiểm tra xem liệu màu của rượu (đỏ hoặc trắng) có ảnh hưởng nhiều đến quality hay ko?

In [ ]:
sns.boxplot(x="type",y="quality",data=df, palette="dark")
plt.show()

=> nhận thấy rằng type của rượu k ảnh hưởng gì nhiều đến quality của nó, ta có thể drop cột này.

Cùng nhìn qua phân bố của các class

In [ ]:
fig = plt.figure(figsize = (18, 4))
title = fig.suptitle("Wine Type Vs Quality", fontsize=14)
fig.subplots_adjust(top=0.85, wspace=0.3)

ax1 = fig.add_subplot(1,4, 1)
ax1.set_title("Red Wine")
ax1.set_xlabel("Quality")
ax1.set_ylabel("Frequency") 
rw_q = df.quality[df.type == 'red'].value_counts()
rw_q = (list(rw_q.index), list(rw_q.values))
ax1.set_ylim([0, 2500])
ax1.tick_params(axis='both', which='major', labelsize=8.5)
bar1 = ax1.bar(rw_q[0], rw_q[1], color='red', edgecolor='black', linewidth=1)


ax2 = fig.add_subplot(1,4, 2)
ax2.set_title("White Wine")
ax2.set_xlabel("Quality")
ax2.set_ylabel("Frequency") 
ww_q = df.quality[df.type == 'white'].value_counts()
ww_q = (list(ww_q.index), list(ww_q.values))
ax2.set_ylim([0, 2500])
ax2.tick_params(axis='both', which='major', labelsize=8.5)
bar2 = ax2.bar(ww_q[0], ww_q[1], color='white', edgecolor='black', linewidth=1)


In [ ]:
#Kiểm tra outlier của data
# tạo box plot
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20,10))
index = 0
ax = ax.flatten()

for col, value in df.items():
    if col != 'type':
        sns.boxplot(y=col, data=df, color='r', ax=ax[index])
        index += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid")
fig, ax1 = plt.subplots(3,4, figsize=(24,30))
k = 0
columns = list(df.columns)
for i in range(3):
    for j in range(4):
            sns.boxplot(df['quality'], df[columns[k]], ax = ax1[i][j], palette='pastel')
            k += 1
plt.show()

Nhận xét: hầu hết các trường đều có outlier! 

In [ ]:
#Sử dụng heat map để tìm correlation
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), annot=True, linewidths=2)

- Ma trận heatmap là cách tốt nhất để thấy được mối quan hệ giữa các trường số với nhau.
- Mỗi tọa độ của ma trận thể hiện mối tương quan giữa 2 cột.
- Correlation càng gần 1, thì 2 trường càng có sự giống nhau.
- Nếu các trường (khác target) có correlation cao, có thể xem xét bỏ đi một cột.
- Nếu một trường có correlation cao với target, thì trường đó được xem xét như một feature quan trọng.

1 vài insights từ bảng trên:
- tỉ lệ alcohol trong rượu đồng biến với quality => alcohol nhiều thì rượu chất lượng càng cao
- alcohol và ph có mối liên hệ yếu, không đáng kể
- Citric acid và density có mối tương quan khá mạnh với fixed acidity.
- pH nghịch biến với density, fixed acidity, citric acid, and sulfates.

In [ ]:
plt.figure(figsize=(15,15))
df.corr().quality.apply(lambda x: abs(x)).sort_values(ascending=False).iloc[1:11][::-1].plot(kind='barh',color='pink') 
# calculating the top 10 highest correlated features
# with respect to the target variable i.e. "quality"
plt.title("Top 10 highly correlated features", size=20, pad=26)
plt.xlabel("Correlation coefficient")
plt.ylabel("Features")

In [ ]:
### Đánh giá alcohol vs quality
sns.boxplot(x='quality', y='alcohol', data = df)

- Các chấm đen thể hiện đó là outlier, chủ yếu ở những chai rượu có quality = 5, ta có thể bỏ qua outlier bằng cách thêm argument showoutliers=False  

In [ ]:
sns.boxplot(x='quality', y='alcohol', data = df, showfliers=False)

=> Nồng độ alcohol cao => chất lượng rượu càng cao

In [ ]:
# Xem phân bố của cột alchohol
sns.distplot(df['alcohol'])

Ta có thể thấy rằng phần bố của alcohol skew với chất lượng của rượu. Kiểm tra mức độ skew của phân bố sử dụng scipy.stats

In [ ]:
from scipy.stats import skew
skew(df['alcohol'])

In [ ]:
## Vẽ phân bố của tất cả các field trong tập dữ liệu
plt.figure(figsize=(15,8))
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20,10))
index = 0
ax = ax.flatten()

for col, value in df.items():
    if col != 'type':
        sns.distplot(value, color='r', ax=ax[index])
        index += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

- Các hình trên biểu diễn phân bố của từng cột dữ liệu
- Có các features có phân bố chuẩn, còn lại chủ yếu là right skew distribution. Range của từng feature cũng ko quá rộng
- Ta cần transform các skewed feature, sử dụng log transfrom sẽ giải quyết được vấn đề!

### <a id='4'> 4. Feature Engineering

1 vài step chúng ta cần xử lí dữ liệu sau khi đã phân tích qua các features:
- Fill null values
- log transform
- rescale data
- xử lí imbalanced data

#### <a id='4.1'> 4.1 Log transform

- ví dụ về output của log_transform

In [ ]:
def log_transform(col):
    return np.log(col)

fixed_acidity_transformed = df[['fixed acidity']].apply(log_transform, axis=1)
chlorides_transformed = df[['chlorides']].apply(log_transform, axis=1)
free_sulfur_dioxide_transformed = df[['free sulfur dioxide']].apply(log_transform, axis=1)
sulphates_transformed = df[['sulphates']].apply(log_transform, axis=1)

In [ ]:
## Vẽ phân bố của tất cả các field trong tập dữ liệu
sns.distplot(fixed_acidity_transformed)

In [ ]:
sns.distplot(chlorides_transformed)

In [ ]:
sns.distplot(free_sulfur_dioxide_transformed)

In [ ]:
sns.distplot(sulphates_transformed)

In [ ]:
df.corr()['quality'].sort_values(ascending=False)

- correlation giữa các feature với target không cao lắm, nhưng có thể thấy các feature quan trọng là alcohol, density, volatile acidity

In [ ]:
# hàm xử lí log_transform
def log_transform(col):
    return np.log(col[0])

df['fixed acidity']= df[['fixed acidity']].apply(log_transform, axis=1)
df['chlorides'] = df[['chlorides']].apply(log_transform, axis=1)
df['free sulfur dioxide'] = df[['free sulfur dioxide']].apply(log_transform, axis=1)
df['sulphates'] = df[['sulphates']].apply(log_transform, axis=1)

#### <a id='4.2'> 4.2 Xử lí imbalanced data

![home price](./imgs/smote.png)

- The simplest implementation of over-sampling is to duplicate random records from the minority class, which can cause overfishing.



- In under-sampling, the simplest technique involves removing random records from the majority class, which can cause loss of information.

In [ ]:
df_3 = df[df.quality==3]     # MINORITY          
df_4 = df[df.quality==4]     # MINORITY          
df_5 = df[df.quality==5]     # MAJORITY
df_6 = df[df.quality==6]     # MAJORITY
df_7 = df[df.quality==7]     # MINORITY
df_8 = df[df.quality==8]     # MINORITY
df_9 = df[df.quality==9]     # MINORITY

In [ ]:
# Oversample MINORITY Class to make balance data :
from sklearn.utils import resample

df_3_upsampled = resample(df_3, replace=True, n_samples=2000) 
df_4_upsampled = resample(df_4, replace=True, n_samples=2000) 
df_7_upsampled = resample(df_7, replace=True, n_samples=2000) 
df_8_upsampled = resample(df_8, replace=True, n_samples=2000) 
df_9_upsampled = resample(df_9, replace=True, n_samples=2000) 
# Decreases the rows of Majority one's to make balance data :
df_5_downsampled = df[df.quality==5].sample(n=2000).reset_index(drop=True)
df_6_downsampled = df[df.quality==6].sample(n=2000).reset_index(drop=True)

In [ ]:
# Combine downsampled majority class with upsampled minority class
Balanced_df = pd.concat([df_3_upsampled, df_4_upsampled, df_7_upsampled, 
                         df_8_upsampled, df_5_downsampled, df_6_downsampled, df_9_upsampled ]).reset_index(drop=True)


# Display new class counts
Balanced_df.quality.value_counts()           

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='quality', data=Balanced_df, order=[3, 4, 5, 6, 7, 8, 9], palette='pastel')

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(x='quality', y = 'alcohol', data = df, palette = 'coolwarm')

In [ ]:
plt.figure(figsize=(15,15))
Balanced_df.corr().quality.apply(lambda x: abs(x)).sort_values(ascending=False).iloc[1:11][::-1].plot(kind='barh',color='pink') 
# calculating the top 10 highest correlated features
# with respect to the target variable i.e. "quality"
plt.title("Top 10 highly correlated features", size=20, pad=26)
plt.xlabel("Correlation coefficient")
plt.ylabel("Features")

- Không làm thay đổi về correlation giữa Feature và target!

#### <a id='4.3'>4.3 Fill null values

##### Xử lí missing data
Có thể nhận thấy số lượng missing value khá ít, nên ta có thể fill các giá trị này bằng 1 số cách như sau:
- Fill với mean: fill các giá trị null với giá trị mean của phân bố, đối với các skewed feature, sử dụng mean để fill các giá trị null có thể làm sai phân bố dữ liệu
- Fill với median: fill các giá trị null với giá trị median của phân bố, sử dụng giá trị median này không làm thay đổi phân bố dữ liệu 
- Fill với giá trị có frequency nhiều nhất trong cột đó
- Fill với 1 giá trị cố định: 0 hoặc 1.

Trong trường hợp missing value quá nhiều thì sẽ không thể fill ngay, sẽ làm sai phân bố của dữ liệu từ đó làm sai việc thuật toán, tốt nhất nên trace xem lí do missing và có thể fill bằng cách nào? Khi việc fill giá trị missing trở nên risky thì nên drop luôn cột đó đi :) 

In [ ]:
## Hàm xử lí missing value
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """
        Impute missing values:
        - Columns of dtype object are imputed with the most frequent value in column.
        - Columns of other types are imputed with mean of column.
        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

Việc scaler chúng ta sẽ sử dụng StandardScaler có sẵn trong sklearn cho việc rescale lại các giá trị trong dataset

In [ ]:
from sklearn.model_selection import train_test_split

cols = Balanced_df.columns
cols = list(cols.drop(['type','quality']))
y=Balanced_df["quality"]
X_train, X_test, y_train, y_test = train_test_split(Balanced_df.loc[:, cols], y, test_size=0.33, random_state=12)

In [ ]:
X_train.describe()

### <a id='5'> 5. Modeling and evaluating for regression:

#### Hồi quy tuyến tính

- Khi sử dụng hồi quy tuyến tính, mục tiêu của chúng ta là để làm sao một đường thẳng có thể tạo được sự phân bố gần nhất với hầu hết các điểm. Do đó làm giảm khoảng cách (sai số) của các điểm dữ liệu cho đến đường đó.
- Trong không gian hai chiều, một hàm số được gọi là tuyến tính nếu đồ thị của nó có dạng một đường thẳng. Trong không gian ba chiều, một hàm số được goi là tuyến tính nếu đồ thị của nó có dạng một mặt phẳng. Trong không gian nhiều hơn 3 chiều, khái niệm mặt phẳng không còn phù hợp nữa, thay vào đó, một khái niệm khác ra đời được gọi là siêu mặt phẳng (hyperplane).

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
LR = Pipeline([
        ('imputer', DataFrameImputer()),
        ('scl', StandardScaler()),
        ('lr',  LinearRegression())
 ])  

LR.fit(X_train,y_train)
y_pred = LR.predict(X_test)

In [ ]:
x = LR.predict(X_train)

In [ ]:
print("Train rmse: " + str(mean_squared_error(y_train, x)**0.5))
print("Test rmse: " + str(mean_squared_error(y_test, y_pred)**0.5))

- so với scale 0-10, thì RMSE ở đây không lớn
- Rmse bộ test nhỏ hơn bộ train => có thể không bị overfit

#### <a id='5.1'> 5.1 Ordinal Classification với 10 classes sử dụng K nearest neightbors

- Giới thiệu về KNN

KNN (K-Nearest Neighbors) là một thuật toán đơn giản nhất trong nhóm thuật toán Học có giám sát.  Ý tưởng của thuật toán này đó là tìm output của một dữ liệu mới dựa trên output của K điểm gần nhất xung quanh nó. KNN được ứng dụng nhiều trong khai phá dữ liệu và học máy. Trong thực tế, việc đo khoảng cách giữa các điểm dữ liệu, chúng ta có thể sử dụng rất nhiều độ đo, tiêu biểu như là  manhattan, euclide, cosine,…

In [ ]:
Image(url='./imgs/knn.png', width=400)

Ví dụ như hình trên, để gán nhãn cho điểm dữ liệu hình sao, ta xét K = 3 điểm gần nhất xung quanh nó. Nhận thấy trong 3 điểm đó, có 2 điểm thuộc class B và 1 điểm thuộc class A. Như vậy ta sẽ gán nhãn cho điểm hình sao sẽ thuộc về class B

Thuật toán của KNN có thể được mô tả như sau:

Thuật toán:
- Xác định tham số K số làng giềng gần nhất
- Tính khoảng cách của đối tượng cần phân lớp tới tất cả các đối tượng có trong tập train
- Lấy top K cho giá trị nhỏ nhất (hoặc lớn nhất)
- Trong top K giá trị vừa lấy, ta thống kê số lượng của mỗi lớp, chọn phân lớp cho số lượng lớn nhất

Vậy K bằng bao nhiêu thì tốt ? ta sẽ cần phải thực nghiệm nhiều lần, và chọn K sao cho kết quả output là tốt nhất

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = Pipeline([
        ('imputer', DataFrameImputer()),
        ('scl', StandardScaler()),
        ('clf',  KNeighborsClassifier())
 ])  

KNN.fit(X_train,y_train)
y_pred = KNN.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
# ~> classification với 10 classes cho ra điểm khá thấp

In [ ]:
# For weights = 'uniform'
for n_neighbors in [5,10,15,20]:
    KNN = Pipeline([
        ('imputer', DataFrameImputer()),
        ('scl', StandardScaler()),
        ('clf',  KNeighborsClassifier(n_neighbors=n_neighbors))
     ])  
    KNN.fit(X_train, y_train) 
    scr = KNN.score(X_test, y_test)
    print("For n_neighbors = ", n_neighbors  ," score is ",scr)

In [ ]:
# For weights = 'distance'
for n_neighbors in [5,10,15,20]:
    KNN = Pipeline([
        ('imputer', DataFrameImputer()),
        ('scl', StandardScaler()),
        ('clf',  KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance'))
     ])  
    KNN.fit(X_train, y_train) 
    scr = KNN.score(X_test, y_test)
    print("For n_neighbors = ", n_neighbors  ," score is ",scr)

#### <a id='5.2'> 5.2 Classification với 3 classes

In [ ]:
Balanced_df['quality_label'] = Balanced_df.quality.apply(lambda q: 'bad' if q <= 5 else 'good' if q <= 7 else 'excellent')

In [ ]:
cols = Balanced_df.columns
cols = list(cols.drop(['type', 'quality_label','quality']))
y=Balanced_df["quality_label"]
X_train, X_test, y_train, y_test = train_test_split(Balanced_df.loc[:, cols], y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
KNN = Pipeline([
        ('imputer', DataFrameImputer()),
        ('scl', StandardScaler()),
        ('clf',  KNeighborsClassifier())
 ])  

KNN.fit(X_train,y_train)
y_pred = KNN.predict(X_test)

labels = np.unique(y_test)
print(classification_report(y_test,y_pred))

cf_matrix_df =  confusion_matrix(y_test, y_pred, labels=labels)

pd.DataFrame(cf_matrix_df, index=labels, columns=labels)